In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import datetime
from scipy import stats

import seaborn as sns
sns.set_context('notebook')
%config InlineBackend.figure_format = 'retina'

In [2]:
full_df=pd.read_pickle('../datasets/df_userID.pickle')

In [3]:
full_df=full_df[full_df.Weather.notnull()]

In [4]:
full_df.head()

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,UserID
534409,"Zermatt Marathon, Zermatt",sam. 05.07.2008,2008,7,M45,42.195,Abgottspon Helmut,M,1962.0,Staldenried,165,06:10:37.200000,00:08:47,Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...,Abgottspon Helmut 1962.0
534410,"Zermatt Marathon, Zermatt",sam. 05.07.2008,2008,7,F55,42.195,Abgottspon Julia,F,1950.0,Thun,5,05:34:08.200000,00:07:55,Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...,Abgottspon Julia 1950.0
534411,"Zermatt Marathon, Zermatt",sam. 05.07.2008,2008,7,M65,42.195,Abgottspon Medard,M,1942.0,Visp,2,05:13:54.700000,00:07:26,Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...,Abgottspon Medard 1942.0
534412,"Zermatt Marathon, Zermatt",sam. 05.07.2008,2008,7,M50,42.195,Achterberg Nico,M,1957.0,Veenendaal,80,05:38:35,00:08:01,Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...,Achterberg Nico 1957.0
534413,"Zermatt Marathon, Zermatt",sam. 05.07.2008,2008,7,M50,42.195,Ackeret Peter,M,1957.0,Deitingen,11,04:26:03.500000,00:06:18,Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...,Ackeret Peter 1957.0


In [5]:
full_df.Race=full_df.Race.apply(lambda x: 'Lucerne Marathon, Luzern' if x=='SwissCityMarathon - Lucerne, Luzern' else x)

# Categorize weather

In [31]:
full_df.Weather.value_counts().index

Index(['Partly cloudy', 'Clear', 'Sunny', 'Mist', 'Patchy rain possible',
       'Cloudy', 'Light rain', 'Light rain shower', 'Moderate rain',
       'Moderate or heavy rain shower', 'Overcast', 'Fog',
       'Moderate rain at times', 'Light drizzle', 'Light snow',
       'Patchy light drizzle', 'Moderate snow', 'Patchy light rain',
       'Patchy light rain with thunder', 'Freezing fog', 'Heavy snow',
       'Heavy rain', 'Moderate or heavy rain with thunder',
       'Moderate or heavy sleet', 'Patchy moderate snow',
       'Heavy rain at times', 'Light sleet',
       'Moderate or heavy snow with thunder'],
      dtype='object')

2 -> Sunny
1 -> Cloudy
0 -> bad weather

In [32]:
dict_favorable_weather={'Partly cloudy':1, 'Clear':2, 'Sunny':2, 'Mist':0, 'Patchy rain possible':1,
       'Cloudy':1, 'Light rain':0, 'Light rain shower':0, 'Moderate rain':0,
       'Overcast':1, 'Fog':0, 'Moderate or heavy rain shower':0,
       'Moderate rain at times':1, 'Light drizzle':0, 'Light snow':0,
       'Patchy light drizzle':0, 'Moderate snow':0, 'Patchy light rain':0,
       'Patchy light rain with thunder':0, 'Freezing fog':0, 'Heavy snow':0,
       'Heavy rain':0, 'Moderate or heavy rain with thunder':0,
       'Moderate or heavy sleet':0, 'Patchy moderate snow':0,
       'Heavy rain at times':0, 'Light sleet':0,
       'Moderate or heavy snow with thunder':0}

In [ ]:
def categorize_weather(x):
    return dict_favorable_weather[x]

In [ ]:
full_df.Weather=full_df.Weather.apply(categorize_weather)

# Races with a lot of years

In [74]:
distance_to_analyse=21.0975

In [75]:
good_distances=full_df[(full_df.Distance==distance_to_analyse)]# | (full_df.Distance==21.0975) | (full_df.Distance==42.195)]

In [76]:
a=good_distances.groupby([good_distances.Race,good_distances.RaceYear]).count().reset_index()

In [77]:
a=a[['Race','RaceYear','Date']]
a=a.rename(columns={'Date':'Runners'})
a['Years']=1
a

,Race,RaceYear,Runners,Years
0,"Aletsch-Halbmarathon, Bettmeralp",2009,1536,1
1,"Aletsch-Halbmarathon, Bettmeralp",2010,1847,1
2,"Aletsch-Halbmarathon, Bettmeralp",2011,1896,1
3,"Aletsch-Halbmarathon, Bettmeralp",2012,2120,1
4,"Aletsch-Halbmarathon, Bettmeralp",2015,2252,1
5,"Bieler Lauftage, Biel/Bienne",2009,513,1
6,"Bieler Lauftage, Biel/Bienne",2015,655,1
7,"Brienzerseelauf, Bönigen",2015,222,1
8,"Bündner Frühlingslauf, Domat / Ems",2011,279,1
9,"Bündner Frühlingslauf, Domat / Ems",2012,197,1


In [78]:
race_stats=a.groupby(a.Race).sum()

In [79]:
race_stats

,RaceYear,Runners,Years
Race,,,
"Aletsch-Halbmarathon, Bettmeralp",10057,9651,5
"Bieler Lauftage, Biel/Bienne",4024,1168,2
"Brienzerseelauf, Bönigen",2015,222,1
"Bündner Frühlingslauf, Domat / Ems",4023,476,2
"Dreiländer-Lauf, Basel",12069,3441,6
Frauenfelder,16092,8140,8
"Graubünden Marathon/Walking, Lenzerheide",2014,119,1
"Hallwilerseelauf, Beinwil am See",14084,24721,7
IWB Basel Marathon,8050,4288,4


In [80]:
race_stats=race_stats[race_stats.Years>5]
race_stats['RunnersPerYear']=race_stats.Runners/race_stats.Years
race_stats=race_stats[race_stats.RunnersPerYear>500]
race_stats

/home/stefano/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,RaceYear,Runners,Years,RunnersPerYear
Race,,,,
"Dreiländer-Lauf, Basel",12069,3441,6,573.500000
Frauenfelder,16092,8140,8,1017.500000
"Hallwilerseelauf, Beinwil am See",14084,24721,7,3531.571429
Lausanne Marathon,12075,17661,6,2943.500000
"Lucerne Marathon, Luzern",14084,41093,7,5870.428571
"Maratona Ticino, Tenero",12063,5349,6,891.500000
"Sempacherseelauf, Sursee",12075,4632,6,772.000000
Winterthur Marathon,14084,8466,7,1209.428571


In [81]:
good_games=race_stats.index
good_games

Index(['Dreiländer-Lauf, Basel', 'Frauenfelder',
       'Hallwilerseelauf, Beinwil am See', 'Lausanne Marathon',
       'Lucerne Marathon, Luzern', 'Maratona Ticino, Tenero',
       'Sempacherseelauf, Sursee', 'Winterthur Marathon'],
      dtype='object', name='Race')

# Example with a distance

In [82]:
reduced_df=full_df[full_df.Race.isin(good_games)]

In [83]:
reduced_df.shape

(173820, 19)

In [84]:
reduced_df.RaceYear.value_counts()

2014    27845
2013    27298
2015    26534
2011    25513
2012    24384
2010    23987
2009    16768
2008     1491
Name: RaceYear, dtype: int64

In [85]:
km10=reduced_df[reduced_df.Distance==distance_to_analyse]

In [86]:
km10.Race.value_counts()

Lucerne Marathon, Luzern            41093
Hallwilerseelauf, Beinwil am See    24721
Lausanne Marathon                   17661
Winterthur Marathon                  8466
Frauenfelder                         8140
Maratona Ticino, Tenero              5349
Sempacherseelauf, Sursee             4632
Dreiländer-Lauf, Basel               3441
Name: Race, dtype: int64

In [87]:
km10.columns

Index(['Race', 'Date', 'RaceYear', 'RaceMonth', 'Category', 'Distance', 'Name',
       'Sex', 'Year', 'LivingPlace', 'Rank', 'Time', 'Pace', 'Place',
       'MinTemp', 'MaxTemp', 'Weather', 'RaceID', 'UserID'],
      dtype='object')

In [88]:
km10=km10[['Race', 'Date', 'RaceYear', 'RaceMonth', 'Category', 'Distance',
       'Sex', 'Time', 'Pace',
       'MinTemp', 'MaxTemp', 'Weather', 'RaceID', 'UserID']]

In [89]:
def add_mean_std_race(x):
    x['MeanRace']=np.mean(x.Time)
    x['StdRace']=np.std(x.Time)/np.sqrt(x.shape[0])
    x['Count']=x.shape[0]
    return x

In [90]:
def add_mean_multiple_years(x):
    x['MeanMultipleYears']=np.mean(x.Time)
    return x

In [91]:
km10=km10.groupby(km10.RaceID).apply(add_mean_std_race)
km10=km10.groupby(km10.Race).apply(add_mean_multiple_years)
km10=km10[km10.Sex=='M']

In [92]:
km10['DiffFromGlobalMean']=km10.MeanRace-km10.MeanMultipleYears
km10.DiffFromGlobalMean=km10.DiffFromGlobalMean.apply(lambda x: x.total_seconds())

In [93]:
km10['AvgTemp']=(km10.MinTemp+km10.MaxTemp)/2

In [94]:
km10

,Race,Date,RaceYear,RaceMonth,Category,Distance,Sex,Time,Pace,MinTemp,MaxTemp,Weather,RaceID,UserID,MeanRace,StdRace,Count,MeanMultipleYears,DiffFromGlobalMean,AvgTemp
569743,Frauenfelder,dim. 16.11.2008,2008,11,HMM60,21.0975,M,01:52:15.300000,00:05:19,-1.0,9.0,Sunny,http://services.datasport.com/2008/waffenlauf/...,Gfr Adank Mathis 1941.0,01:43:17.272914,00:00:34.215081,875,01:43:44.376547,-27.103634,4.0
569744,Frauenfelder,dim. 16.11.2008,2008,11,HMM40,21.0975,M,01:43:39.100000,00:04:54,-1.0,9.0,Sunny,http://services.datasport.com/2008/waffenlauf/...,Herr Aerne Jürg 1962.0,01:43:17.272914,00:00:34.215081,875,01:43:44.376547,-27.103634,4.0
569745,Frauenfelder,dim. 16.11.2008,2008,11,HMM40,21.0975,M,01:39:22.400000,00:04:42,-1.0,9.0,Sunny,http://services.datasport.com/2008/waffenlauf/...,Pal Sdt Aeschbach Hansruedi 1963.0,01:43:17.272914,00:00:34.215081,875,01:43:44.376547,-27.103634,4.0
569746,Frauenfelder,dim. 16.11.2008,2008,11,HMM40,21.0975,M,02:03:47,00:05:51,-1.0,9.0,Sunny,http://services.datasport.com/2008/waffenlauf/...,Afshar Manochehr 1961.0,01:43:17.272914,00:00:34.215081,875,01:43:44.376547,-27.103634,4.0
569747,Frauenfelder,dim. 16.11.2008,2008,11,HMM30,21.0975,M,01:28:54.600000,00:04:12,-1.0,9.0,Sunny,http://services.datasport.com/2008/waffenlauf/...,Gfr Akermann Elmar 1970.0,01:43:17.272914,00:00:34.215081,875,01:43:44.376547,-27.103634,4.0
569750,Frauenfelder,dim. 16.11.2008,2008,11,HMM40,21.0975,M,01:36:42.900000,00:04:35,-1.0,9.0,Sunny,http://services.datasport.com/2008/waffenlauf/...,Alig Urs 1966.0,01:43:17.272914,00:00:34.215081,875,01:43:44.376547,-27.103634,4.0
569751,Frauenfelder,dim. 16.11.2008,2008,11,HMM30,21.0975,M,02:06:20.400000,00:05:59,-1.0,9.0,Sunny,http://services.datasport.com/2008/waffenlauf/...,Altorfer Martin 1976.0,01:43:17.272914,00:00:34.215081,875,01:43:44.376547,-27.103634,4.0
569752,Frauenfelder,dim. 16.11.2008,2008,11,HMM30,21.0975,M,01:30:33,00:04:17,-1.0,9.0,Sunny,http://services.datasport.com/2008/waffenlauf/...,Altorfer Oliver 1969.0,01:43:17.272914,00:00:34.215081,875,01:43:44.376547,-27.103634,4.0
569755,Frauenfelder,dim. 16.11.2008,2008,11,HMM50,21.0975,M,01:46:44.600000,00:05:03,-1.0,9.0,Sunny,http://services.datasport.com/2008/waffenlauf/...,Ambühl Beat 1953.0,01:43:17.272914,00:00:34.215081,875,01:43:44.376547,-27.103634,4.0
569757,Frauenfelder,dim. 16.11.2008,2008,11,HMM60,21.0975,M,01:51:51.300000,00:05:18,-1.0,9.0,Sunny,http://services.datasport.com/2008/waffenlauf/...,Ammann Anton 1939.0,01:43:17.272914,00:00:34.215081,875,01:43:44.376547,-27.103634,4.0


In [95]:
km10.columns

Index(['Race', 'Date', 'RaceYear', 'RaceMonth', 'Category', 'Distance', 'Sex',
       'Time', 'Pace', 'MinTemp', 'MaxTemp', 'Weather', 'RaceID', 'UserID',
       'MeanRace', 'StdRace', 'Count', 'MeanMultipleYears',
       'DiffFromGlobalMean', 'AvgTemp'],
      dtype='object')

In [96]:
del final_data

In [100]:
final_data=km10.groupby(km10.RaceID).first().reset_index()
final_data=final_data[['Race', 'RaceYear','Distance',
       'Weather', 'AvgTemp', 'DiffFromGlobalMean',
       'MeanRace', 'MeanMultipleYears',
        'StdRace', 'Count', 'MinTemp', 'MaxTemp','RaceID' ]]
final_data['CatWeather']=final_data.Weather.apply(lambda x: dict_favorable_weather[x])
final_data

,Race,RaceYear,Distance,Weather,AvgTemp,DiffFromGlobalMean,MeanRace,MeanMultipleYears,StdRace,Count,MinTemp,MaxTemp,RaceID,CatWeather
0,Frauenfelder,2008,21.0975,Sunny,4.0,-27.103634,01:43:17.272914,01:43:44.376547,00:00:34.215081,875,-1.0,9.0,http://services.datasport.com/2008/waffenlauf/...,2
1,"Dreiländer-Lauf, Basel",2009,21.0975,Clear,18.0,141.089126,01:52:25.097524,01:50:04.008398,00:00:39.422019,606,12.0,24.0,http://services.datasport.com/2009/lauf/dreila...,2
2,"Hallwilerseelauf, Beinwil am See",2009,21.0975,Light rain shower,11.5,-0.973756,01:50:09.217454,01:50:10.191209,00:00:15.929069,3661,9.0,14.0,http://services.datasport.com/2009/lauf/hallwil,0
3,"Lucerne Marathon, Luzern",2009,21.0975,Partly cloudy,8.5,5.490592,01:53:50.149779,01:53:44.659187,00:00:14.112650,4998,4.0,13.0,http://services.datasport.com/2009/lauf/lucern...,1
4,"Maratona Ticino, Tenero",2009,21.0975,Patchy light rain,1.0,-78.358856,01:44:45.203062,01:46:03.561918,00:00:29.215492,1045,0.0,2.0,http://services.datasport.com/2009/lauf/tenero,0
5,Winterthur Marathon,2009,21.0975,Clear,23.0,186.896396,01:54:33.225678,01:51:26.329281,00:00:31.889579,1106,16.0,30.0,http://services.datasport.com/2009/lauf/winter...,2
6,Frauenfelder,2009,21.0975,Partly cloudy,8.0,-19.072362,01:43:25.304186,01:43:44.376547,00:00:36.237899,860,4.0,12.0,http://services.datasport.com/2009/waffenlauf/...,1
7,"Dreiländer-Lauf, Basel",2010,21.0975,Partly cloudy,10.5,-161.925300,01:47:22.083098,01:50:04.008398,00:00:34.475022,568,8.0,13.0,http://services.datasport.com/2010/lauf/dreila...,1
8,Frauenfelder,2010,21.0975,Partly cloudy,2.5,-56.007867,01:42:48.368681,01:43:44.376547,00:00:34.645088,910,-1.0,6.0,http://services.datasport.com/2010/lauf/frauen...,1
9,"Hallwilerseelauf, Beinwil am See",2010,21.0975,Light rain,3.0,-69.413506,01:49:00.777703,01:50:10.191209,00:00:16.404276,3588,-1.0,7.0,http://services.datasport.com/2010/lauf/hallwil,0


In [101]:
final_data

,Race,RaceYear,Distance,Weather,AvgTemp,DiffFromGlobalMean,MeanRace,MeanMultipleYears,StdRace,Count,MinTemp,MaxTemp,RaceID,CatWeather
0,Frauenfelder,2008,21.0975,Sunny,4.0,-27.103634,01:43:17.272914,01:43:44.376547,00:00:34.215081,875,-1.0,9.0,http://services.datasport.com/2008/waffenlauf/...,2
1,"Dreiländer-Lauf, Basel",2009,21.0975,Clear,18.0,141.089126,01:52:25.097524,01:50:04.008398,00:00:39.422019,606,12.0,24.0,http://services.datasport.com/2009/lauf/dreila...,2
2,"Hallwilerseelauf, Beinwil am See",2009,21.0975,Light rain shower,11.5,-0.973756,01:50:09.217454,01:50:10.191209,00:00:15.929069,3661,9.0,14.0,http://services.datasport.com/2009/lauf/hallwil,0
3,"Lucerne Marathon, Luzern",2009,21.0975,Partly cloudy,8.5,5.490592,01:53:50.149779,01:53:44.659187,00:00:14.112650,4998,4.0,13.0,http://services.datasport.com/2009/lauf/lucern...,1
4,"Maratona Ticino, Tenero",2009,21.0975,Patchy light rain,1.0,-78.358856,01:44:45.203062,01:46:03.561918,00:00:29.215492,1045,0.0,2.0,http://services.datasport.com/2009/lauf/tenero,0
5,Winterthur Marathon,2009,21.0975,Clear,23.0,186.896396,01:54:33.225678,01:51:26.329281,00:00:31.889579,1106,16.0,30.0,http://services.datasport.com/2009/lauf/winter...,2
6,Frauenfelder,2009,21.0975,Partly cloudy,8.0,-19.072362,01:43:25.304186,01:43:44.376547,00:00:36.237899,860,4.0,12.0,http://services.datasport.com/2009/waffenlauf/...,1
7,"Dreiländer-Lauf, Basel",2010,21.0975,Partly cloudy,10.5,-161.925300,01:47:22.083098,01:50:04.008398,00:00:34.475022,568,8.0,13.0,http://services.datasport.com/2010/lauf/dreila...,1
8,Frauenfelder,2010,21.0975,Partly cloudy,2.5,-56.007867,01:42:48.368681,01:43:44.376547,00:00:34.645088,910,-1.0,6.0,http://services.datasport.com/2010/lauf/frauen...,1
9,"Hallwilerseelauf, Beinwil am See",2010,21.0975,Light rain,3.0,-69.413506,01:49:00.777703,01:50:10.191209,00:00:16.404276,3588,-1.0,7.0,http://services.datasport.com/2010/lauf/hallwil,0


In [105]:
final_data.groupby(final_data.Race).first()

,RaceYear,Distance,Weather,AvgTemp,DiffFromGlobalMean,MeanRace,MeanMultipleYears,StdRace,Count,MinTemp,MaxTemp,RaceID,CatWeather
Race,,,,,,,,,,,,,
"Dreiländer-Lauf, Basel",2009,21.0975,Clear,18.0,141.089126,01:52:25.097524,01:50:04.008398,00:00:39.422019,606,12.0,24.0,http://services.datasport.com/2009/lauf/dreila...,2
Frauenfelder,2008,21.0975,Sunny,4.0,-27.103634,01:43:17.272914,01:43:44.376547,00:00:34.215081,875,-1.0,9.0,http://services.datasport.com/2008/waffenlauf/...,2
"Hallwilerseelauf, Beinwil am See",2009,21.0975,Light rain shower,11.5,-0.973756,01:50:09.217454,01:50:10.191209,00:00:15.929069,3661,9.0,14.0,http://services.datasport.com/2009/lauf/hallwil,0
Lausanne Marathon,2010,21.0975,Partly cloudy,7.5,-128.337606,01:47:52.988655,01:50:01.326261,00:00:19.545733,2803,3.0,12.0,http://services.datasport.com/2010/lauf/lamara,1
"Lucerne Marathon, Luzern",2009,21.0975,Partly cloudy,8.5,5.490592,01:53:50.149779,01:53:44.659187,00:00:14.112650,4998,4.0,13.0,http://services.datasport.com/2009/lauf/lucern...,1
"Maratona Ticino, Tenero",2009,21.0975,Patchy light rain,1.0,-78.358856,01:44:45.203062,01:46:03.561918,00:00:29.215492,1045,0.0,2.0,http://services.datasport.com/2009/lauf/tenero,0
"Sempacherseelauf, Sursee",2010,21.0975,Patchy rain possible,11.0,-104.152593,01:43:20.150817,01:45:04.303411,00:00:27.135632,856,7.0,15.0,http://services.datasport.com/2010/lauf/sempac...,1
Winterthur Marathon,2009,21.0975,Clear,23.0,186.896396,01:54:33.225678,01:51:26.329281,00:00:31.889579,1106,16.0,30.0,http://services.datasport.com/2009/lauf/winter...,2


In [103]:
np.corrcoef(final_data.MinTemp,final_data.DiffFromGlobalMean)

array([[ 1.        ,  0.51364916],
       [ 0.51364916,  1.        ]])

In [99]:
compact=km10.groupby(km10.Race).first()
compact=compact[['Distance','MeanRace']]
compact

,Distance,MeanRace
Race,,
"Dreiländer-Lauf, Basel",21.0975,01:52:25.097524
Frauenfelder,21.0975,01:43:17.272914
"Hallwilerseelauf, Beinwil am See",21.0975,01:50:09.217454
Lausanne Marathon,21.0975,01:47:52.988655
"Lucerne Marathon, Luzern",21.0975,01:53:50.149779
"Maratona Ticino, Tenero",21.0975,01:44:45.203062
"Sempacherseelauf, Sursee",21.0975,01:43:20.150817
Winterthur Marathon,21.0975,01:54:33.225678
